In [12]:
from numpy.random import RandomState
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import jieba
import jieba.analyse
import re

In [13]:
df = pd.read_csv('movies.csv')
#str -> dict
df['類別'] = df['類別'].apply(eval)
# list first catagory
def list_genres(x):
    for d in x.values():
        return(d)
        
df['主類別'] = df['類別'].apply(list_genres)

In [14]:
#split overview
def split_overview(df):
    overview = []
    for d in df['劇情介紹']:
        keywords = jieba.analyse.extract_tags(d, topK=5)
        overview.append(keywords)

    df['關鍵字'] = overview
    return df['關鍵字']

In [15]:
df['關鍵字'] = split_overview(df)

In [16]:
df['keywords'] = ''
def bag_words(x):
    return (''.join(re.sub('[^\u4e00-\u9fa5]+', '', x['中文名稱'])) + ' ' + ''.join(x['主類別']) + ' ' +  ' '.join(x['關鍵字']))
df['keywords'] = df.apply(bag_words, axis = 1)

In [17]:
#split Data to train and test
rng = RandomState()
train = df.sample(frac=0.9173, random_state=rng)
test = df.loc[~df.index.isin(train.index)]
#test['主類別'].to_csv('test_Data.csv')

In [18]:
# 將文件中的詞語轉換為詞頻矩陣
cv = CountVectorizer()
cv_train = cv.fit_transform(train['keywords'])
cv_test = cv.transform(test['keywords'])

In [19]:
#類調用
transformer = TfidfTransformer()
#將詞頻矩陣X統計成TF-IDF值
tfidf_train = transformer.fit_transform(cv_train)
tfidf_test = transformer.transform(cv_test)

In [20]:
knn  = KNeighborsClassifier(n_neighbors=11,p=2,n_jobs=2)
knn.fit(tfidf_train,train['主類別'])

KNeighborsClassifier(n_jobs=2, n_neighbors=11)

In [21]:
#predict test_Data
knn.predict(tfidf_test)

array(['動作', '動作', '動作', '動作', '動畫', '動畫', '動作', '動作', '動作', '劇情', '動作',
       '動作', '劇情', '劇情', '動作', '動作', '動作', '動畫', '動作', '動作', '劇情', '動作',
       '動作', '動作', '恐怖', '動作', '動作', '動作', '動作', '動作', '動作', '戰爭', '戰爭',
       '動作', '動作', '動作', '動作', '動作', '動作', '動作', '動作', '動作', '劇情', '動作',
       '動作', '動作', '動作', '恐怖', '動作', '動作', '動作', '動作', '劇情', '動作', '動作',
       '動作', '劇情', '動作', '動作', '動作', '恐怖', '動作', '動作', '動作', '戰爭', '動作',
       '動作', '動作', '動作', '動作', '動作', '動作', '動作', '動畫', '動畫', '動畫', '動作',
       '冒險', '動畫', '冒險', '動作', '冒險', '冒險', '動作', '動作', '冒險', '奇幻', '冒險',
       '冒險', '冒險', '動作', '動作', '劇情', '冒險', '冒險', '冒險', '溫馨/家庭', '劇情',
       '動作', '冒險', '動作', '劇情', '動作', '動作', '動作', '冒險', '劇情', '動作', '冒險',
       '動作', '溫馨/家庭', '動作', '動作', '動作', '科幻', '劇情', '科幻', '恐怖', '科幻',
       '劇情', '動作', '動作', '科幻', '劇情', '劇情', '動作', '科幻', '動作', '動作', '冒險',
       '劇情', '動作', '劇情', '愛情', '溫馨/家庭', '奇幻', '愛情', '奇幻', '動畫', '動畫',
       '奇幻', '奇幻', '奇幻', '劇情', '奇幻', '劇情', '劇情', '溫馨/家庭', '劇

In [22]:
#精準度%
Accuracy = knn.score(tfidf_test,test['主類別'])
print(Accuracy*100 , '%')

76.6 %
